rmt lk 184 kuni 187.
Näidiskood: https://github.com/PacktPublishing/TinyML-Cookbook/blob/main/Chapter05/ColabNotebooks/prepare_model.ipynb

Tauno Erik 14.08.2022

# Train model

0. Import libraries

In [ ]:
import numpy as np
import pathlib
import tensorflow as tf
import tensorflow_datasets as tfds
import zipfile
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Model

1. Constants

In [ ]:
MODEL_ALPHA = 0.35
MODEL_INPUT_WIDTH = 48
MODEL_INPUT_HEIGHT = 48
TFL_MODEL_FILE = "scene_recognition.tflite"
TFL_MODEL_HEADER_FILE = "scene_recognition_model.h"
TF_MODEL = "scene_recognition"

## Transfer learning with Keras

2. Onen in Colab and upload dataset.zip to root directory. (For me it does not work!)
Or download dataset directly from GitHub:

In [ ]:
!wget "https://github.com/taunoe/flower-camera/raw/main/dataset.zip"


3. Unzip the dataset:

In [ ]:
with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
  zip_ref.extractall(".")

data_dir = "dataset"

4. Resizes the input images to 48x48 with the bilinear interpolation. Prepare the training (80%) and validation (20%) datsets:

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)) # const

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT))

5. Get the name of the classes

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

6. Rescale the pixel values from [0, 255] to [-1, 1] because the pre-trained model expects this interval data range for the input tensor.

In [ ]:
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds = val_ds.map(lambda x, y: (rescale(x), y))

7. Import the MobileNet v2 pre-trained model with the weights trained on the ImageNet dataset and alpha=0.35. Furthermore, set the input image at the lowest resolution allowed by the pre-trained model (48, 48, 3) and exclude the top (fully-connected) layers:

In [ ]:
base_model = MobileNetV2(input_shape=(48, 48, 3),
                         include_top=False,
                         weights='imagenet',
                         alpha=0.35)

8. Freeze the weights so that you do not update these values during training:

In [ ]:
base_model.trainable = False
feat_extr = base_model

9. Augment the input data:

In [ ]:
augmen = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

train_ds = train_ds.map(lambda x, y: (augmen(x), y))
val_ds = val_ds.map(lambda x, y: (augmen(x), y))

10. Prepare the classification head with a global pooling followed by a dense layer with a softmax activation:

In [ ]:
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()

dense_layer = tf.keras.layers.Dense(num_classes,activation='softmax')

11. Build the model architecture:

In [ ]:
inputs = tf.keras.Input(shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3))
x = global_avg_layer(feat_extr.layers[-1].output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = dense_layer(x)
model = tf.keras.Model(inputs=feat_extr.inputs,outputs=outputs)

12. Compile the model with a 0.0005 learning rate:

In [ ]:
lr = 0.0005

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

13. Model summary

In [ ]:
model.summary()

14. Train the model with 10 epochs:

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10)

15.  Save the TensorFlow model:

In [ ]:
model.save(TF_MODEL)

## Preparing and testing the quantized TFLite model

16. Unzip the test dataset (test_samples.zip)

In [ ]:
!wget "https://github.com/taunoe/flower-camera/raw/main/test_samples.zip"

In [ ]:
with zipfile.ZipFile("test_samples.zip", 'r') as zip_ref:
    zip_ref.extractall(".")
    
test_dir = "dataset"

17. Rescale the pixel values from [0,255] to [-1,1]

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                      interpolation="bilinear",
                                                      image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT))
test_ds  = test_ds.map(lambda x, y: (rescale(x), y))

18. Quantize the TensorFlow model with the TFLite converter to TensorFlow Lite format (FlatBuffers). Apply the 8-bit quantization to the entire model except for the output layer:

In [ ]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]

converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)
converter.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8

tfl_model = converter.convert()

19. Get the TFLite model size in bytes

In [ ]:
size_tfl_model = len(tfl_model)
print(len(tfl_model), "bytes")

20. Initialize the TFLite interpreter

In [ ]:
# Initialize the TFLite interpreter
interpreter = tf.lite.Interpreter(model_content=tfl_model)

# Allocate the tensors
interpreter.allocate_tensors()

21. Get input quantization parameters

In [ ]:
# Get input/output layer information
i_details = interpreter.get_input_details()[0]
o_details = interpreter.get_output_details()[0]

# Get input quantization parameters.
i_quant = i_details["quantization_parameters"]
i_scale      = i_quant['scales'][0]
i_zero_point = i_quant['zero_points'][0]

22. Evaluate the accuracy of the quantized TFLite model

In [ ]:
test_ds0 = val_ds.unbatch()

num_correct_samples = 0
num_total_samples   = len(list(test_ds0.batch(1)))

for i_value, o_value in test_ds0.batch(1):
  i_value = (i_value / i_scale) + i_zero_point
  i_value = tf.cast(i_value, dtype=tf.int8)
  interpreter.set_tensor(i_details["index"], i_value)
  interpreter.invoke()
  o_pred = interpreter.get_tensor(o_details["index"])[0]

  if np.argmax(o_pred) == o_value:
    num_correct_samples += 1

print("Accuracy:", num_correct_samples/num_total_samples)

23. Convert the TFLite model to C-byte array with xxd

In [ ]:
open("model.tflite", "wb").write(tfl_model)
!apt-get update && apt-get -qq install xxd
!xxd -c 60 -i model.tflite > scene_recognition.h

Wait and download the scene_recognition.h file from Colab's left pane.

The end!